# Two body system

Consider a system of two bodies under gravitational attraction. The masses are $m_1,m_2$ and the gravitational constant is $\mathcal{G}$.

The generalised coordinates of the system are the polar coordinates $r,\theta$ of the body of mass $m_1$ in the reference frame centred on the centre of gravity $O$ of the two bodies ($O$ itself has a rectilinear uniform motion). The standard (Cartesian) coordinates are then given by

\begin{equation*}
x = r\cos\theta
\hspace{2cm}
y = r\sin\theta
\end{equation*}

By construction, the polar coordinates of the other body are $\frac{m_1}{m_2}r,-\theta$, and the distance between the two bodies is therefore given by $r(1+\frac{m_1}{m_2})$. Hence, the gravitational force exerted on $m_1$ (by $m_2$) is

\begin{equation*}
F = -\mathcal{G}\frac{m_1m_2}{(1+\frac{m_1}{m_2})^2r^2}
\left(\begin{array}{l}\cos\theta\\ \sin\theta\end{array}\right) = -\nabla V
\hspace{1cm}\textrm{with}\hspace{1cm}
V \triangleq -\mathcal{G}\frac{m_1m_2}{(1+\frac{m_1}{m_2})^2}\frac{1}{r}
\end{equation*}

Hence the Lagrangian

\begin{equation*}
\mathcal{L} = m_1(\frac{1}{2}(\dot{r}^2+r^2\dot{\theta}^2)+\frac{a}{r})
\hspace{1cm}\textrm{with}\hspace{1cm}
a \triangleq \frac{\mathcal{G}m_2}{(1+\frac{m_1}{m_2})^2}
\end{equation*}

Introducing the intermediary variable $\lambda=r^2\dot{\theta}$ (angular momentum), the equations of the dynamics are given by:

\begin{equation*}
\left\{
\begin{array}{rrcl}
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\theta}} = \frac{\partial\mathcal{L}}{\partial\theta}\right]
\hspace{1cm} &
\dot{\theta} & = & \frac{\lambda}{r^2}\\
& \dot{\lambda} & = & 0\\
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{r}} = \frac{\partial\mathcal{L}}{\partial r}\right]
\hspace{1cm} &
\ddot{r} & = & \frac{\lambda^2}{r^3}-\frac{a}{r^2}
\end{array}
\right.
\end{equation*}

Documentation: [here](../doc/_build/html/odesimu.html)

In [1]:
%pylab notebook
# for external animation (less resource consuming), use appropriate backend, e.g. qt5

import logging; logging.basicConfig(level='WARN') # can be changed dynamically by logger_hook
from functools import partial
from ipyshow import Setup
from ipyshow.odesimu import System
from ipyshow.odesimu.util import logger_hook

Populating the interactive namespace from numpy and matplotlib


## G2body class

In [2]:
#----------------------------------------------------------------------------------------------------
class G2body (System):
#----------------------------------------------------------------------------------------------------

  shadowshape = (4,)

  @Setup(
      'M1,M2: mass of the bodies [kg]',
      'G: gravitational constant [m^3.kg^-1.sec^-2]',
  )
  def __init__(self,M1,M2,G):
    self.M1,self.M2,self.G = M1, M2, G
    self.a = G*M2/square(1+M1/M2)
    def main(t,state,a=self.a):
      r,θ,rʹ,λ = state
      r2 = square(r)
      θʹ = λ/r2
      rʺ = (square(λ)/r-a)/r2
      λʹ = 0.
      return array((rʹ,θʹ,rʺ,λʹ))
    self.main = main
    def jac(t,state,a=self.a):
      r,θ,rʹ,λ = state
      r2 = square(r)
      r3 = r*r2
      return array((
        (0,0,1,0),
        (-2*λ/r3,0,0,1/r2),
        ((-3*square(λ)/r+2*a)/r3,0,0,2*λ/r3),
        (0,0,0,0),
        ))
    self.jacobian = jac
    def fordisplay(state,t=M1/M2):
      r,θ,rʹ,λ = state
      x,y = r*cos(θ),r*sin(θ)
      live = x,y,-t*x,-t*y
      return live, live
    self.fordisplay = fordisplay

  def display(self,ax,refsize=100.,ini=None,**ka):
    from matplotlib.patches import Ellipse
    μ = self.M1/self.M2
    ratio = clip(sqrt(μ),1./refsize,refsize)
    sz = (refsize*ratio,refsize/ratio)
    diag_l, = ax.plot((),(),'k',ls=':')
    diag_s = ax.scatter((0,0),(0,0),s=sz,marker='o',c=('r','b'))
    tail_l = ax.plot((),(),'r',(),(),'b')
    p,e,ψ,E,λ = self.analytics(ini)
    centre = e*self.a*array((cos(ψ),sin(ψ)))/(2*E)
    T = pi*self.a*(-E)**-1.5/(2*sqrt(2)) if E<0 else None
    def focus_():
      ax.relim()
      ax.autoscale_view()
    focus = lambda x,y: focus_()
    def prefocus(m=.1): ax.margins(m); focus_(); return lambda x,y: None
    if λ==0:
      trajectory = 'Straight line segment'
      if E<0:
        trajectory += f' $T$={T:.2f}'
        apex = self.a/E*array((cos(ψ),sin(ψ)))
        ax.plot(*zip(apex,-μ*apex),c='k',marker='o',ls='none')
        focus = prefocus()
    elif abs(E)<1e-10:
      trajectory = 'Parabola'
    elif E<0:
      A,B = self.a/(-2*E),λ/sqrt(-2*E)
      trajectory = f'Ellipse($e$={e:.2f}) half-period:{T:.2f}'
      ax.add_patch(Ellipse(centre,2*A,2*B,ψ/pi*180,lw=1,color='k',fill=False,ls='dashed'))
      ax.add_patch(Ellipse(-centre*μ,2*A*μ,2*B*μ,ψ/pi*180,visible=False))
      focus = prefocus()
    else: # E>0
      ψₒ = arccos(-1/e)
      asymp_l, = ax.plot((),(),'k',ls='--')
      trajectory = f'Hyperbola($e$={e:.2f})'
      def focus(x,y,a=array(((sin(ψ-ψₒ),-cos(ψ-ψₒ)),(sin(ψ+ψₒ),-cos(ψ+ψₒ))))):
        z = array((x,y))
        d = a@(z-centre)
        i = 0 if abs(d[1])>abs(d[0]) else 1
        asymp_l.set_data(*zip(centre,z-d[i]*a[i]))
        focus_()
    ax.set_title(f'trajectory: {trajectory}')
    def disp(t,live,tail):
      x1,y1,x2,y2 = live
      diag_l.set_data((x1,x2),(y1,y2))
      diag_s.set_offsets(((x1,y1),(x2,y2)))
      tail_l[0].set_data(tail[:,0],tail[:,1])
      tail_l[1].set_data(tail[:,2],tail[:,3])
      focus(x1,y1)
    return super().display(ax,disp,ini=ini,**ka)

  def analytics(self,ini):
    r,θ,rʹ,λ = ini
    E = .5*(square(rʹ)+square(λ/r))-self.a/r
    p = square(λ)/self.a
    e = sqrt(1+2*E*p/self.a)
    φ = arcsin(rʹ*λ/(self.a*e))
    if p<r: φ=pi-φ
    ψ = θ-φ
    return p,e,ψ,E,λ

  @staticmethod
  @Setup(
    'r: distance of 1st body from centre of gravity [m]',
    'θ: angle of axis between the two bodies with reference [deg]',
    'rʹ: speed of 1st body towards (or away from) centre of gravity [m.sec^-1]',
    'θʹ: angular speed [deg.sec^-1]',
    θ=0.,rʹ=0.,θʹ=0.)
  def makestate(r,θ,rʹ,θʹ): return array((r,radians(θ),rʹ,square(r)*radians(θʹ)))

  @Setup(
    System.launch,
    'refsize: average size (area) of the bodies for display [pt^2]',
    taild=10.,hooks=(partial(logger_hook,logger=logging.getLogger()),)
  )
  def launch(self,*a,**ka): return super().launch(*a,**ka)

In [3]:
Setup.display(G2body)

**** G2body.__init__(self, M1, M2, G) ****
    M1,M2     : mass of the bodies [kg]
    G         : gravitational constant [m^3.kg^-1.sec^-2]
**** G2body.launch(self, *a, **ka) ****
    ini       : initial state []
    srate     (25.0      ): sampling rate [sec^-1]
    maxtime   (inf       ): total simulation time length [sec]
    taild     (10.0      ): shadow duration [sec]
    listeners : listener binding (events: start,stop,step,error) []
    hooks     ((functools): list of display hooks []
    refsize   : average size (area) of the bodies for display [pt^2]
**** G2body.makestate(r, θ, rʹ, θʹ) ****
    r         : distance of 1st body from centre of gravity [m]
    θ         (0.0       ): angle of axis between the two bodies with reference [deg]
    rʹ        (0.0       ): speed of 1st body towards (or away from) centre of gravity [m.sec^-1]
    θʹ        (0.0       ): angular speed [deg.sec^-1]

In [4]:
#syst = G2body(G=.1,M1=10,M2=400)
syst = G2body(G=1,M1=10,M2=80)
#syst = G2body(G=1,M1=10,M2=10)
ini = dict(r=15.,rʹ=-1.,θ=30.,θʹ=3.) # elipse
#ini = dict(r=15.,rʹ=-5.,θ=30.,θʹ=3.) # hyperbola
#ini = dict(r=15.,rʹ=0.,θ=30.,θʹ=sqrt(2*syst.a)/(15**1.5)*180/pi) # parabola
#ini = dict(r=15.,rʹ=.7,θ=30.,θʹ=0) # straight line
syst.launch(ini=syst.makestate(**ini))

<IPython.core.display.Javascript object>

## Trajectory

Multiplying the (last) Lagrangian Equation by $\dot{r}$, then integrating, yields $\frac{1}{2}(\dot{r}^2+\frac{\lambda^2}{r^2})-\frac{a}{r}=E$ for some constant $E$ (total energy). Hence

\begin{equation*}
\dot{r}^2 = 2E+\frac{2a}{r}-\frac{\lambda^2}{r^2}
\hspace{2cm}
\dot{\theta} = \frac{\lambda}{r^2}
\end{equation*}

Case $\dot{\theta}_o=0$
----

In that case $\lambda=0$ and $\dot{\theta}=0$ hence $\theta=\theta_o$ and the trajectory is a straight line going through the origin with angle $\theta_o$. The equation for $r$ is $\dot{r}^2=2(E+\frac{a}{r})$, which becomes, incorporating the initial conditions:

\begin{equation*}
\dot{r}^2 = 2a(\frac{1}{r}-\frac{q}{r_o})
\hspace{1cm}\textrm{where}\hspace{1cm}
q\triangleq -\frac{r_oE}{a} = 1-\frac{r_o\dot{r}_o^2}{2a}
\\
\textrm{hence}\hspace{1cm}
\mathbf{d}t=\frac{\epsilon}{\sqrt{2a}}(\frac{1}{r}-\frac{q}{r_o})^{-\frac{1}{2}}\mathbf{d}r
\hspace{1cm}\textrm{for some}\hspace{1cm}
\epsilon\in\{-1,1\}
\end{equation*}

When $q>0$, this has an analytic solution given by:
\begin{equation*}
\mathbf{d}t = -\epsilon\sqrt{\frac{r_o^3}{2aq^3}} \mathbf{d}\left[\sqrt{\frac{qr}{r_o}(1-\frac{qr}{r_o})}+\arctan\sqrt{\frac{r_o}{qr}-1}\right]
\end{equation*}

* If $\dot{r}_o\geq0$ and $q>0$ i.e. $r_o\dot{r}_o^2<2a$, the mass first moves away from the centre ($\epsilon=1$) and reaches the inversion point $r^*\triangleq\frac{r_o}{q}$ at $T^*$, then moves towards the centre ($\epsilon=-1$) and reaches it at $T^*+T$.
* If $\dot{r}_o\geq0$ and $q\leq0$ i.e. $r_o\dot{r}_o^2\geq2a$, the mass moves away from the centre ($\epsilon=1$) forever.
* If $\dot{r}_o<0$, the mass moves towards the centre ($\epsilon=-1$). If $q>0$, it reaches it at $T(-\dot{r}_o)-T^*(-\dot{r}_o)$.

Once at the centre, the trajectory is undefined (collision!): the solver should fail. By integration (case $q>0$):

\begin{equation*}
T^* = \sqrt{\frac{r_o^3}{2aq^3}}\left[q\sqrt{\frac{1}{q}-1}+\arctan\sqrt{\frac{1}{q}-1}\right]
\hspace{2cm}
T = \frac{\pi}{2\sqrt{2a}}\left(\frac{r_o}{q}\right)^{\frac{3}{2}} = \frac{\pi a}{2\sqrt{2}}(-E)^{-\frac{3}{2}}
\end{equation*}

Case $\dot{\theta}_o\not=0$
----

In that case $\lambda\not=0$. Hence $\theta$ is a monotonous function of time (since  $\dot{\theta}=\frac{\lambda}{r^2}\not=0$) hence $r$ can be expressed as a function of $\theta$ and $\frac{\mathbf{d}r}{\mathbf{d}\theta}=\frac{\dot{r}}{\dot{\theta}}$. One gets

\begin{equation*}
(\frac{\mathbf{d}r}{\mathbf{d}\theta})^2 = r^4(\frac{2E}{\lambda^2}+\frac{2a}{\lambda^2r}-\frac{1}{r^2})
\hspace{1cm}\textrm{with initial conditions}\hspace{1cm}
r|_{\theta_0} = r_0
\textrm{ and }
\frac{\mathbf{d}r}{\mathbf{d}\theta}|_{\theta_0}=\frac{\dot{r}_0r_0^2}{\lambda}
\end{equation*}

Hence

\begin{equation*}
(\frac{\mathbf{d}\frac{1}{r}}{\mathbf{d}\theta})^2
= (\frac{1}{r^2}\frac{\mathbf{d}r}{\mathbf{d}\theta})^2
= \frac{2E}{\lambda^2}+\frac{2a}{\lambda^2}\frac{1}{r}-\frac{1}{r^2}
= \frac{2E}{\lambda^2}+\frac{a^2}{\lambda^4}-(\frac{1}{r}-\frac{a}{\lambda^2})^2
= (\frac{e}{p})^2-(\frac{1}{r}-\frac{1}{p})^2
\end{equation*}

where
\begin{equation*}
p\triangleq\frac{\lambda^2}{a}
\textrm{ and }
e\triangleq\sqrt{1+\frac{2E\lambda^2}{a^2}}
\end{equation*}

One solution is given by

\begin{equation*}
r = \frac{p}{1+e\cos(\theta-\phi)}
\hspace{1cm}\textrm{with}\hspace{1cm}
\left|
\begin{array}{l}
\sin(\theta_0-\phi)\triangleq\frac{\lambda\dot{r}_0}{ae}\\
\cos(\theta_0-\phi)\triangleq\frac{p-r_0}{er_0}
\end{array}
\right.
\end{equation*}

The presence of two (consistent) initial conditions compensate for the unknown sign of  $\frac{\mathbf{d}r}{\mathbf{d}\theta}$, so that the system is deterministic (unique solution), hence the trajectory above is the only possible one. It is a conic section (ellipse if $e\lt1$, parabola if $e=1$, hyperbola if $e\gt1$) with the origin $O$ as focus. The initial conditions $\theta_o,\dot{\theta}_o,r_o,\dot{r}_o$ determine the values of $\lambda,E$, and hence those of $p,e,\phi$ (by construction, $\phi$ is unique modulo $2\pi$). 

Since $\dot{\theta}\not=0$, time can be seen as a function of $\theta$, hence of $u=\tan\frac{\theta-\phi}{2}$

\begin{equation*}
\mathbf{d}t = \frac{r^2}{\lambda}\mathbf{d}\theta =
\frac{p^2}{\lambda(1+e\cos(\theta-\phi))^2}\mathbf{d}\theta =
\frac{2p^2\mathbf{d}u}{\lambda(1+u^2)(1+e\frac{1-u^2}{1+u^2})^2} =
\frac{2p^2}{\lambda(1-e)^2} \frac{1+u^2}{(c+u^2)^2}\mathbf{d}u
\hspace{1cm}\textrm{with}\hspace{1cm}c \triangleq \frac{1+e}{1-e}
\end{equation*}

Simplifying the algebraic fraction in the right-hand side, we get, in the elliptical case $e<1$ (hence $c>0$)

\begin{equation*}
\mathbf{d}t =
\frac{2p^2}{\lambda(1-e)^2}\left(\frac{1-c}{2c}\frac{c-u^2}{(c+u^2)^2}+\frac{1+c}{2c}\frac{1}{c+u^2}\right)\mathbf{d}u =
\frac{2p^2}{\lambda(1-e)^2(1+e)}\mathbf{d}\left[-\frac{eu}{c+u^2}+\frac{1}{\sqrt{c}}\arctan\frac{u}{\sqrt{c}}\right]
\end{equation*}

The motion is periodic and the half-period $T$ is given by integrating for $\theta$ in the range $[\phi,\phi+\pi]$ (or $[\phi-\pi,\phi]$), i.e. $u$ in the range $[0,+\infty]$:

\begin{equation*}
T = \frac{\pi}{\sqrt{a}} \left(\frac{p}{1-e^2}\right)^{\frac{3}{2}} = \frac{\pi a}{2\sqrt{2}}(-E)^{-\frac{3}{2}}
\end{equation*}